# Good Trackpad!

See the README.md for project description

From the 5-jupyter things article

First maintain your imports in one cell so you can quickly see dependencies

Oh, and notice the heading?  You can getting some visual order to our docs using markdown. [Jupyter markdown cheat sheet](http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

A few notes on import.

The statement `import library as localname` brings a code library into your application namespace.  The `localname` saves effort when you use the library's objects and their methods in your code.

Here's a quick overview of these libraries, taken from [the SciPy quick start](https://www.scipy.org/getting-started.html), with a bullet on Panda's added for convenience:
* NumPy‘s array type augments the Python language with an efficient data structure useful for numerical work, e.g., manipulating matrices. NumPy also provides basic numerical routines, such as tools for finding eigenvectors.
* SciPy contains additional routines needed in scientific work: for example, routines for computing integrals numerically, solving differential equations, optimization, and sparse matrices.
* Matplotlib produces high quality plots. With it you can turn your data or your models into figures for presentations or articles. No need to do the numerical work in one program, save the data, and plot it with another program.
* Pandas provides fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data both easy and intuitive. Elements can be of different data types. There are lots of useful utilies for reading, cleaning and manipulating data. Think very powerful spreadsheets.

This summary is good enough for now, so feel free to skip ahead. But, you can only put off learning the details of your data structures for so long.

If you're coming from Matlab your may find [Numpy for Matlab users](https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html) helpful both for quick orientation to numpy and clarity on python.

When you can't grok the code anymore, you've hit the point where you need to learn more.  [Linus Torvalds famously said](https://softwareengineering.stackexchange.com/questions/163185/torvalds-quote-about-good-programmer) "Bad programmers worry about the code. Good programmers worry about data structures and their relationships."  Knowing how your data is structured will answer the programming question "what next?" Change your data so it meets your next need.   

When you need the details, the [numpy quickstart](https://docs.scipy.org/doc/numpy-dev/user/quickstart.html) provides a stable foundation.  

Numpy provides the ndarray data type, an enhanced array data type for efficient numerical operations. The ndarray is an homogeneous, multidimensional arrays with methods to inspect its structure.  The ndarry provides better performance for numeric operations and more efficient notation for coding.

Here's a list of features:
* homogenous, multi dimensional by default
* ndarray is the python data type
* index is a tuple of the number of elements that are the dimensions of the ndarray
* each tuple entry is a reference to one of the dimensions in the ndarray separated by commas with colon range operators allowed
* each dimension is an axis
* the number of axis is the rank (note the linear algebra term gives a hint of numpy's focus)
* an ndarray is created from standard arrays
* ndarray's can be pre-created in any dimension to avoid expensive resizes during construction
* arange and linspace generate ranges of integer and floating point numbers
* printing ndarrays uses a short-hand for skipping interior elements in the output display
* arithmatic with ndarrays is always element-wise
* common arithmetic operations on matrix are included as methods.  eg. A.sum()

The [main pandas docs page](https://pandas.pydata.org/pandas-docs/stable/index.html) gives a great overview and feeling of where panda's plays a role in the work of a data scientist (hint: everywhere). [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html) is very action oriented.

Set up inline graphics

In [ ]:
# Will allow us to embed images in the notebook
%matplotlib inline
# change default plot size
plt.rcParams['figure.figsize'] = (15,10)

The data import is easy with pandas cause this is how it reads a csv.

We can cut the header of our cnee file with a simple `grep Cypress goodmouse-2017-10-28-1.dat > cnee.csv` then read that file

In [ ]:
trackpad = pd.read_csv("cnee.csv", header=None)

You can always bring up help for a methon with the `help()` function, like `help(pd.read_csv)`

Take a look at the data just to make sure its what we think it is

In [ ]:
trackpad

A the plot method is part of all data object types and is a quick way to look at the data.  A simple data point plot can be drawn with the 'o' style.

In [ ]:
trackpad[2].plot(style='o')

What we really need is to see this data in 2D since we are mapping a 2D surface.  This means we want the X values on the x-axis and Y as the y-axis.  Just looking at the raw data let's us see where all the trackpad was touched.

It doesnt' give us any insight into when or what kind of trajectories we developed here but this is where we can start to use our clustering algos to see how closely together the interaction occurred

In [ ]:
plt.scatter(trackpad[2],trackpad[3])

Let's look at some longer running input and see if more of the traces can be seen.

In [ ]:
tp2=pd.read_csv("cnee2.csv", header=None)

plt.scatter(tp2[2],tp2[3])

Our goal is to understand this data and get a sense both of what it means and if there's any pattern we can detect with inputs to distinguish as real and which are stray input from my hands touching the trackpad when i type.

The goal is to use [Scikit-learn](http://scikit-learn.org) for data discovery.  There is a very helpful [cheat sheet that maps out options](http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) for data discovery based on the need to classify or predict values for a data set.

What's not clear is how to read this data.  It's time stamped so even the density is above is miss leading.

One way to think about this is that it is a collection of traces on the trackpad.  It would be good to know how many traces exists.  That is, to seperate the input into distinct time events that identify individual traces.   We could then compare the shapes of the traces and see if there's anything particular about intential action and the stray actions that muck with the trackpad.

A few things we can guess intuitively, if we just masked off a triangular area in the uppper left and right of the trackpad and ignored all input there, we might go a long way to reducing the errors with little effort.  Just place some non-conductive tape over the corners. Presto!

The down sides are, this solves the problem the easy way and we want to solve it the clever way.  Your hands can feel the tape boundary. The tape edges will likey gather lint and will eventually curl. You'll have to repeat the fix at some regular interval.  It lacks style.  It detracts from the appearance of the machine.

Letood 's set out to find how we can separate our data.  Asking google about [analyzing path data sets](https://www.google.com/search?client=ubuntu&channel=fs&q=analyzing+path+data+sets&ie=utf-8&oe=utf-8) doesn't seem to go in the direction we are going. 

It seems reasonable that things that happen close together in time are related.  Things that trace a geometric shape or some line would be more likely intentional than some random walk or touch points.

How do we identify these data boundaries?  Also, why do the random touch points seem to happen randomly. Or rather sometimes this problem is really a non-issue.

Tuning our questions to google to get more specific, we want to [isolate events in time series data](https://www.google.com/search?client=ubuntu&channel=fs&q=isolating+events+in+time+series+data&ie=utf-8&oe=utf-8).  This brings up a paper on [real-time pattern isolation and recognition](http://infolab.usc.edu/DocsDemos/mmm03-final.pdf) describing sensing user input and islating events in a virtual environemnt.  This feels more like the direction to head in.

There's a discussion thread of someone asking basically the same question [detecting events in time series data](https://cs.stackexchange.com/questions/61102/detecting-events-in-time-series-data). However, this discussion is focused on continuous signal time series data.

In our data set we don't have a continuous signal, data just appears when it occurs. This means we can easily detect the existance of "signal". It's just there.  What we really want is to separate our signal in time.  So go from one long stream of data to looking at the shapes of many different entries.

Tuning our question to [event detection in time series data brings up this helpful thread](https://cs.stackexchange.com/questions/57830/machine-learning-identify-patterns-in-time-series-data) a link to a survey paper on techniques.  There is a recent [survey paper on technics of machine learning for time series data](https://www.researchgate.net/publication/260086856_A_Review_of_Unsupervised_Feature_Learning_and_Deep_Learning_for_Time-Series_Modeling).  It suggests time series data analysis is a hard problem.  Dang.

Let's see if can just find the distribution of distances between events.  Maybe this would be helpful.


Column 7 is the data that looks like time stamps.
Pandas has a [diff() method](http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.diff.html) for looking at time differences between events.

In [ ]:
tp2[7].diff()[1:]

This looks hopeful. But no idea if these are seconds or some other value.


In [ ]:
tp2[7].diff()[1:].describe()

So 75% of the data is above 13 seconds  of separation and 25% is less than 12 seconds.  Maybe this is useful.

How can we break or input traces into separate events?  What's the count of these events?


In [ ]:
plt.hist(tp2[7].diff()[1:], bins=4, log=1)

In [ ]:
pd.to_datetime(tp2[7])

So my time series data is not really date values.

Looking at a date in normal unix time `date +%s` shows the values are 1509301203
The dates above are 243084487

In [ ]:
243084487 - 1509301203

So need to understand my data a bit better.

In [ ]:
p2=pd.read_csv("cnee3.csv", header=None)

plt.scatter(tp2[2],tp2[3])

Take a break if things aren't working as expected

It's always good to clear your mind if it feels like you are at a dead end.  I took a run for this one.   Thinking about the data in this last picture, I realized I am getting the x input data.  That means it's mapped to my screen.  It doesn't show what my trackpad is receiving.  It shows what my desktop is recieving.  This means it's capturing, in some ways, more meaningful data.  It's certainly going to see valid input patterns.  You can see them in the verticle lines.  This is where I'm scrolling.   Most of my windows are full screen so literal positions on the screen that recieved mouse input.

The problem is, I don't what the screen's view of the input.  I want the trackpad's perspective.  It's that trackpad postitions that are getting touched once in a while and causing my focus to change to where ever the cursor happens to be.   I want the position on the trackpad location.  This should cause the stray inputs to have a focus around the upper corners.  I'm also like to find most touching in the center.  

Read the man page on the xev tool.  Pay attention when you read these. It's easy to overlook important details when your scanning.   It's for creating a window the prints xevents or debugging the events sent to a window this starts a window and prints events on the stdout of the caller

The stdout looks like this and makes it clear the time field is some sort of time indicator
```
PropertyNotify event, serial 37, synthetic NO, window 0x6e00001,
    atom 0x15d (_NET_WM_STATE), time 265181821, state PropertyNewValue
```

Even thought i don't know what the time represents the difference values are significant as spacing between events.

A second look

Should be able to get a better picture of the input distribution if I eliminate the very large outlier.

The distribution indicates a big gap above about 300k with a vally before the really big delay. Let's cut those outliers out and see what we get.

Pandas makes it easy to [drop data from an array](https://chrisalbon.com/python/pandas_dropping_column_and_rows.html).  We can just say keep only the rows below the cutoff.  Zooming in on this through a few iterations shows the most useful values are below 100 and generally indicates that our associated events can somewhat be considered to fall below that value.

In [ ]:
tpdiff=tp2[7].diff()[1:] # get a dedicated diff array 
tpclean=tpdiff[tpdiff < 100]

plt.hist(tpclean, bins=30, log=1)

It would be better to discover the clusters using some more advance tooling.  Like maybe an clustering algorithm.

What kind of graph does this imply.  The difference is a distance between nodes.  We know a distance below 13 is very likely within a cluster, or sequence of moves.

So all nodes that are below 13 could be considered secquence.  Would clustering help make a better determination? 

Scikit-learn has clustering algorithms. Given the nature of the data it seems that a heirarchical clustering might work.  We can kind of imagine it.  We want it to first connect all the closest entries into groups.  It can stop when reach some defined ideal number of clusters. For example, at a point where the time between input groupings reaches some threashold.

Scikit-learn has an [AgglomerativeClustering](http://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_clustering.html) tool that may help.  In fact, the docs are even set up as python notebook, so we can open it and have active documentaion to explore.  Spending some time in that notebook leads to breaking down the example it some much smaller chunks of 5 to 10 entries to understand. It won't work for us because it still requires specifying a max number of clusters for our grouping.  We want to discover the number of clusters.

BTW, don't hesitate to start up python on the command line to throw out some simple test code. It's a quick way to explore an unfamiliar method like [numpy method concatenate()](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.concatenate.html). Ctrl-d or exit() get you out.

Remember to talk with co-workers to see if they have any hints on how to approach the problem.

This limitation with clustering also suggests we need to learn more about clustering methods, especially for time series data.   [Time-series clustering – A decade review](http://www.sciencedirect.com/science/article/pii/S0306437915000733) and [Hierarchical Clustering of Time-Series Data Streams](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=4407702)

Let's skip that question for now and see if we can break our data structure into individual traces.

A simple hello world trace on the trackpad shows the problem of the input mapping.  The goal was to paint a smiley face on the trackpad.  This should be possible because i can lift my finger and place it in new locations.  But because this is a mouse position trace, I had to provide continuous input to relocate the mouse.  This caused all the face features to have connecting lines.  

It also hightlights something about the data that I hadn't realized, the values I'm plotting are up-side-down.  This "head" was drawn starting the circle at the top, then the eyes and mouth.  The rendered smiley face is flipped.

In [ ]:
tp3=pd.read_csv("hello.csv", header=None)

plt.scatter(tp3[2],tp3[3])

A good reminder that in the world of graphics the (0,0) point is in the upper left and y increases to the bottom of the screen.

It's easy enough to flip visuall with a `y * -1` but that means we should fix the displayed y-axis values.

In [ ]:
plt.scatter(tp3[2],-1 * tp3[3])

Let's iterate over the data set and separate out frames if the time difference is greater than say 20 as a starting point.

In [ ]:
tp2.iloc[1:2]

In [ ]:
maxdiff = 20
curtime = tp2[7][0]
scenestart = 1
scenenum = 1
sceneend = 1
scenes = []

for row in tp2.itertuples(index=True):
    if (tp2[7][row.Index] - curtime) < maxdiff:
        sceneend += 1
        curtime = tp2[7][row.Index]
    else:
        sceneend += 1
    
        scenes.append(tp2.iloc[scenestart:sceneend])
    
        curtime = tp2[7][row.Index]
        scenestart = sceneend
        scenenum += 1

In [ ]:
scenes[11]

In [ ]:
len(scenes)

In [ ]:
lenarray= []

for idx, series in enumerate(scenes):
    lenarray.append(len(scenes[idx]))

Even if you have created a traditional array, go ahead and convert it to series.  The summary display notation itself if worth it, especially if there are lots of elements.

In [ ]:
lenseries = pd.Series(lenarray)

lenseries

In [ ]:
lenseries = pd.Series(lenarray)

In [ ]:
lenseries.describe()

In [ ]:
lenseries.hist()

## Scene selection 

Let's take a look at some of the scenes to see if they make sense.

In [ ]:
scenes[0]

In [ ]:
plt.scatter(scenes[0][2],-1 * scenes[0][3])

It would be really helpful to look at a panel of these scenes side-by-side.  So let's [build some sub plots](https://matplotlib.org/users/pyplot_tutorial.html#working-with-multiple-figures-and-axes).

It would be even cooler to animate these over time. Fortunately, matplotlib supports animations but will have to [explore the example animation codes](https://matplotlib.org/2.0.0/examples/animation/basic_example.html) later.

In [ ]:
for i in range(10):
    print(i)

In [ ]:
plt.figure(1)

for i in range(1,17):
    plt.subplot(4,4,i)
    plt.scatter(scenes[i-1][2],-1 * scenes[i-1][3])

plt.show()

A quick visual comparison between the boundary of less than 100 vs 200 shows we have a pretty good break point.  At least for our needs.  A machine learning approach for feature clustering would be nice but might be premature.  We may even require more, or better, data before we can do this correctly.  Still the above shapes are a start.

Just for fun, we could look at the last 16 entries too.

In [ ]:
for i in range(1,17):
    plt.subplot(4,4,i)
    plt.scatter(scenes[673 + i-1][2],-1 * scenes[673 + i-1][3])

plt.show()